If using google drive please edit this line to connect to drive location

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from google.colab import drive
import os, sys

# Mount the google colab
drive.mount("/content/drive/")
GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'DeepLearning_GroupProject/'
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
!ls {GOOGLE_DRIVE_PATH}
sys.path.append(GOOGLE_DRIVE_PATH)

# relative paths
models_dir = 'models'
csv_path = 'Data/updated_data_with_lists.csv'
tensors_dir = 'Data/tensor_batches'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
 Code  'Image Recipe.gdoc'   models			 __pycache__	     runner.py
 Data   Main.ipynb	    'Project Task List.gsheet'	 recipe_encoder.py   training.ipynb


In [3]:
import pandas as pd
from ast import literal_eval
import numpy as np
import torch

# Explicitly adding models to the search path
models_path = os.path.join(GOOGLE_DRIVE_PATH, models_dir)
if models_path not in sys.path:
    sys.path.insert(0, models_path)

from models import recipe_encoder

csv_file = os.path.join(GOOGLE_DRIVE_PATH,csv_path)
df = pd.read_csv(csv_file, converters={"Cleaned_Ingredients": literal_eval, "Instructions": literal_eval})
print(len(df))
df.head()

13496


,Unnamed: 0,Title,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[Pat chicken dry with paper towels, season all...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa..."
1,1,Crispy Salt and Pepper Potatoes,[Preheat oven to 400°F and line a rimmed bakin...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo..."
2,2,Thanksgiving Mac and Cheese,[Place a rack in middle of oven; preheat to 40...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts..."
3,3,Italian Sausage and Bread Stuffing,[Preheat oven to 350°F with rack in middle. Ge...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int..."
4,4,Newton's Law,[Stir together brown sugar and hot water in a ...,newtons-law-apple-bourbon-cocktail,"[1 teaspoon dark brown sugar, 1 teaspoon hot w..."


Concatenate the batches of preprocessed images into 1 tensor

In [4]:
pt_files = os.listdir(os.path.join(GOOGLE_DRIVE_PATH,tensors_dir))
all_image_tensors = []
all_image_labels = []

# Load and combine all .pt files
for pt_file in pt_files:
    pt_filepath = os.path.join(GOOGLE_DRIVE_PATH,tensors_dir,pt_file)
    image_tensors, image_labels = torch.load(pt_filepath)
    all_image_tensors.append(image_tensors)
    all_image_labels.extend(image_labels)
    print(f"Loaded {pt_file}")

# Concatenate tensors
all_image_tensors = torch.cat(all_image_tensors)
print(f"Number of images: {all_image_tensors.size(0)}")
print(f"Number of labels: {len(all_image_labels)}")
assert all_image_tensors.size(0) == len(all_image_labels), "Mismatch between images and labels!"

<ipython-input-4-5f21046bf71c>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image_tensors, image_labels = torch.load(pt_filepath)


Loaded batch_0.pt
Loaded batch_1.pt
Loaded batch_2.pt
Loaded batch_3.pt
Loaded batch_4.pt
Loaded batch_5.pt
Loaded batch_6.pt
Loaded batch_7.pt
Loaded batch_8.pt
Loaded batch_9.pt
Loaded batch_10.pt
Loaded batch_11.pt
Loaded batch_12.pt
Loaded batch_13.pt
Number of images: 13582
Number of labels: 13582


Reorganize dataframe to be in the same order as the Image Tensors using the image_label as the matching key

In [5]:
##Reset order of dataframe to match the image labels orders
all_image_labels_cleaned = [label.split(".")[0] for label in all_image_labels] #remove the .png and .jgp
print(all_image_labels_cleaned[:10]) #print to see if at the end the df matches

filtered_df = df[df["Image_Name"].isin(all_image_labels_cleaned)] #filter the df to only have values from the images
print(filtered_df["Image_Name"][:10])
print(df.shape)
print(filtered_df.shape, len(all_image_labels_cleaned))

valid_labels = set(filtered_df['Image_Name'])

# Filter labels and tensors
filtered_labels_and_tensors = [
    (tensor, label) for tensor, label in zip(all_image_tensors, all_image_labels_cleaned) if label in valid_labels
]

# Unpack the filtered data
filtered_tensors, filtered_labels = zip(*filtered_labels_and_tensors)

# Convert back to tensors
filtered_tensors = torch.stack(filtered_tensors)
filtered_labels = list(filtered_labels)

# Verify alignment
print(f"Number of filtered tensors: {len(filtered_tensors)}")
print(f"Number of filtered labels: {len(filtered_labels)}")
print(f"Number of rows in filtered_df: {len(filtered_df)}")

# Finally reorganize the df to be in the same order as the image tensors
filtered_df = filtered_df.set_index("Image_Name").loc[filtered_labels].reset_index()
print(filtered_df["Image_Name"][:10])

['spanish-style-grilled-vegetables-with-breadcrumb-picada-238806', 'three-cheese-pizza-with-onion-sage-and-arugula-233543', 'speedy-skillet-ravioli-lasagna', 'summer-anchovy-salad-51108430', 'spice-roasted-cornish-hens-with-cucumber-yogurt-sauce-353418', 'stovetop-butterscotch-apples-and-cranberries', 'tarte-tatin-51196820', 'spanish-olive-and-cream-cheese-canapes-231160', 'spice-rubbed-sustainable-fish-sliders', 'spiced-pumpkin-layer-cake-240123']
0    miso-butter-roast-chicken-acorn-squash-panzanella
1           crispy-salt-and-pepper-potatoes-dan-kluger
2           thanksgiving-mac-and-cheese-erick-williams
3            italian-sausage-and-bread-stuffing-240559
4                   newtons-law-apple-bourbon-cocktail
5                 warm-comfort-tequila-chamomile-toddy
6                      apples-and-oranges-spiked-cider
7                    turmeric-hot-toddy-claire-sprouse
8                              instant-pot-lamb-haleem
9       spiced-lentil-and-caramelized-onion-baked-eg

Reformat Ingredients, Recipes, and Image titles

In [6]:
column = filtered_df["Cleaned_Ingredients"]
# item_list = [word
#                     for w_list in column
#                     for str_list in w_list
#                     # Make period and comma separate words
#                     # # Remove parentheses and quotes
#                     for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()]

item_list = []
element_list = []
ingr_max = 0
print("Column length", len(column))
for w_list in column:
    tmp =[]
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(',' ( ').replace(')',' ) ').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        tmp.extend(formatted)
        if new_len>ingr_max:
            ingr_max=new_len
    element_list.append(tmp)

print(item_list[:10])
print(element_list[0])
print("Len Elements:", len(element_list))
ingr_vocab = set(item_list) #We need to keep these even if they are redundant so image tensors stay matching
ingr_vocab_size = len(ingr_vocab)
print("Size:", ingr_vocab_size)
print("Max:", ingr_max)
print(ingr_vocab)

Column length 13466
['3', 'large', 'red', 'bell', 'peppers', '(', 'about', '1', '1/2', 'pounds']
['3', 'large', 'red', 'bell', 'peppers', '(', 'about', '1', '1/2', 'pounds', ')', ',', 'stemmed', ',', 'seeded', ',', 'quartered', '4', 'large', 'Japanese', 'eggplants', '(', 'about', '1', '1/4', 'pounds', ')', ',', 'trimmed', ',', 'cut', 'lengthwise', 'into', '3', 'slices', '4', 'medium', 'green', 'or', 'yellow', 'zucchini', '(', 'preferably', '2', 'of', 'each;', 'about', '1', 'pound', ')', ',', 'trimmed', ',', 'cut', 'lengthwise', 'into', '1/3-inch-thick', 'slices', 'Extra-virgin', 'olive', 'oil', '(', 'for', 'grilling', ')', '6', 'tablespoons', 'extra-virgin', 'olive', 'oil', ',', 'divided', '2', 'garlic', 'cloves', ',', 'finely', 'chopped', '1/2', 'teaspoon', 'dried', 'crushed', 'red', 'pepper', '1/2', 'cup', 'panko', '(', 'Japanese', 'breadcrumbs', ')', '*', '2', 'tablespoons', 'Sherry', 'wine', 'vinegar', '1/4', 'cup', 'chopped', 'fresh', 'Italian', 'parsley', '2', 'tablespoons', 'cho

In [7]:
column = filtered_df["Instructions"]
# item_list = [word
#                     for w_list in column
#                     for str_list in w_list
#                     # Make period and comma separate words
#                     # # Remove parentheses and quotes
#                     for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()]

item_list = []
inst_max = 0
for w_list in column:
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        if new_len>inst_max:
            inst_max=new_len

print(item_list[:10])
inst_vocab = set(item_list)
inst_vocab_size = len(inst_vocab)
print("Size:",inst_vocab_size)
print("Max:", inst_max)

['Prepare', 'barbecue', 'medium', 'heat', '.', 'Arrange', 'vegetables', 'on', 'baking', 'sheets']
Size: 21148
Max: 247


In [8]:
column = filtered_df["Title"].to_list()
# item_list = [word
#                 for str_list in column
#     #                     # Make period and comma separate words
#     #                     # # Remove parentheses and quotes
#                 for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
#                 if word]

item_list = []
title_max = 0
for str_list in column:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        if new_len>title_max:
            title_max=new_len


print(item_list[:10])
title_vocab = set(item_list)
title_vocab_size = len(title_vocab)
print("Size:", title_vocab_size)
print("Max:", title_max)

['Spanish-Style', 'Grilled', 'Vegetables', 'with', 'Breadcrumb', 'Picada', 'Three-Cheese', 'Pizza', 'with', 'Onion']
Size: 6950
Max: 21


In [9]:
total_vocab = set(list(inst_vocab) + list(ingr_vocab) + list(title_vocab))
total_vocab_size = len(total_vocab)
print("Size:",total_vocab_size)
total_max = max([inst_max, ingr_max, title_max])
print("Max:",total_max)

Size: 31523
Max: 247


Tokenize Recipes, Ingredients, and Image Titles

In [10]:
from collections import Counter
# word_to_ix = {word: i for i, word in enumerate(total_vocab)}

# word_to_ix_ingr = {word: i for i, word in enumerate(ingr_vocab)}
# word_to_ix_inst = {word: i for i, word in enumerate(inst_vocab)}
# word_to_ix_title = {word: i for i, word in enumerate(title_vocab)}

word_to_ix = {word: i for i, (word,empty) in enumerate(Counter(total_vocab).most_common(), start=1)}
word_to_ix['<PAD>'] = 0
print(word_to_ix)

{'shirt': 1, 'bacon-wrapped': 2, 'th': 3, 'Grape-Ricotta': 4, '**Clear': 5, 'saving': 6, 'medium-heat': 7, 'tbsp/15': 8, "there's": 9, 'giganti': 10, 'Nyora': 11, 'Magpie': 12, 'Rock-Shrimp': 13, 'Walnut-Coconut': 14, 'cross-hatch': 15, '118°F': 16, '“plain”': 17, 'rapid-rise': 18, 'underbake': 19, 'Removing': 20, 'spills': 21, 'gratinata': 22, 'high-quality': 23, 'Chokha': 24, 'chana': 25, 'avo': 26, 'tablespoons/11': 27, 'hooks': 28, 'Ian': 29, 'Grass-Fed': 30, 'sprouts**': 31, 'Wedges:': 32, 'Gallete': 33, 'Strength': 34, 'pepper-jack': 35, 'ask': 36, 'riz;': 37, 'plums;': 38, 'Concord': 39, 'Chevre': 40, 'AHEAD:Can': 41, 'Stuffing': 42, 'Converted': 43, 'structure': 44, '6-pound': 45, 'extra-strength': 46, 'Ramos': 47, 'Smoked-Bacon': 48, 'Erawan': 49, 'Simón': 50, 'Bouillot': 51, 'extend': 52, 'Orient': 53, 'Brace': 54, 'St-Germain': 55, 'whoever': 56, 'Silpat': 57, 'dry-cured': 58, 'Clavidor': 59, 'Lager': 60, 'calzones': 61, 'ground-spice': 62, 'bulb': 63, 'rainbow': 64, 'Two-Ch

In [11]:
def tokenize_text(list_object, vocab, max_length):
    tokenized_list = []
    # Tokenize each sentence
    for sentence in list_object:
      formatted = sentence.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
      tokens = [vocab.get(word, vocab['<PAD>']) for word in formatted]
      # Pad or truncate to max_length
      if len(tokens) < max_length:
          tokens.extend([vocab['<PAD>']] * (max_length - len(tokens)))
      else:
          tokens = tokens[:max_length]
      # Append the tokenized sentence to the list
      tokenized_list.append(tokens)

    return tokenized_list
# Tokenize all columns
filtered_df['Title_List'] = df['Title'].apply(lambda x: [x])
filtered_df['tokenized_ingredients'] = filtered_df['Cleaned_Ingredients'].apply(lambda x: tokenize_text(x, word_to_ix,total_max))
filtered_df['tokenized_instructions'] = filtered_df['Instructions'].apply(lambda x: tokenize_text(x, word_to_ix,total_max))
filtered_df['tokenized_titles'] = filtered_df['Title_List'].apply(lambda x: tokenize_text(x, word_to_ix,total_max))
# print(filtered_df['Title_List'])
# print(filtered_df[['tokenized_ingredients', 'tokenized_instructions', 'tokenized_titles', 'Cleaned_Ingredients', 'Instructions', 'Title']])
max_length_ing = filtered_df['tokenized_ingredients'].apply(len).max()
max_length_inst = filtered_df['tokenized_instructions'].apply(len).max()
max_length_title = filtered_df['tokenized_titles'].apply(len).max()
print(max_length_ing, max_length_inst, max_length_title)

51 117 1


In [12]:
def pad_or_truncate_nested(sublist, target_length, max_length, pad_token=0):
        """
            Pad or truncate the outer list of a nested list to match the target_length.
            Each inner list remains untouched.
        """
        # Pad with [pad_token] or truncate the outer list
        if len(sublist) < target_length:
            sublist.extend([[pad_token]* max_length] * (target_length - len(sublist)))
        else:
            sublist = sublist[:target_length]
        return sublist

filtered_df['tokenized_ingredients'] = filtered_df['tokenized_ingredients'].apply(
    lambda ing: pad_or_truncate_nested(ing, max_length_ing,total_max))

filtered_df['tokenized_instructions'] = filtered_df['tokenized_instructions'].apply(
    lambda inst: pad_or_truncate_nested(inst, max_length_inst, total_max))
# new_token_ing = [pad_or_truncate_nested(ing, max_length_title) for ing in tokenized_ingredients] #titles were all list length of 1
max_length_ing = filtered_df['tokenized_ingredients'].apply(len).max()
max_length_inst = filtered_df['tokenized_instructions'].apply(len).max()
max_length_title = filtered_df['tokenized_titles'].apply(len).max()
print(max_length_ing, max_length_inst, max_length_title)
print(len(filtered_df['tokenized_ingredients'][5]))
print(filtered_df['tokenized_ingredients'][100])

51 117 1
51
[[17339, 18155, 16536, 4537, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [9426, 19605, 5193, 1160, 21969, 15472, 1685, 27431, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Tokenize the Image Labels for the Image Encoder

In [13]:
from transformers import AutoTokenizer
from transformers import CLIPModel
model_name = "openai/clip-vit-base-patch16"
clip_model = CLIPModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenized_labels = tokenizer(
    filtered_labels,
    padding="max_length",
    truncation=True,
    max_length=tokenizer.model_max_length,
    return_tensors="pt"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [22]:
from runner import Runner as R
kwargs = {
    'epochs': 10,
    'ingredient_tokens': filtered_df['tokenized_ingredients'].to_list(),
    'instruction_tokens': filtered_df['tokenized_instructions'].to_list(),
    'title_tokens': filtered_df['tokenized_titles'].to_list(),
    'image_tensors': filtered_tensors,
    'image_labels': tokenized_labels,
    'device': device,
    'vocab_size': total_vocab_size,
    'max_len': total_max,
    'clip_model': clip_model,
    'optimizer': 'adam',
    'learning_rate': 1e-3,
    'batch_size': 1
    # 'max_lengths': {
    #     'ingredient_tokens': max_length_ing,
    #     'instruction_tokens': max_length_inst,
    #     'title_tokens': max_length_title
    # }

}
image2recipe = R(**kwargs)
image2recipe.train()


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.ShortTensor instead (while checking arguments for embedding)